## Dog Breed Classification

In this project we will use traditional CNN, CNN with data augmentation and finally transfer Learning by VGG16 model with weights pre-trained on Imagenet to solve the dog breed classification problem

### Load Dataset Files

In [0]:
# Executing the Model Creation from my laptop

In [29]:
import pandas as pd
import os
import cv2
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.image import ImageDataGenerator,img_to_array,load_img
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers.convolutional import Conv2D,MaxPooling2D
from keras.layers.core import Activation,Flatten,Dense,Dropout
from keras.layers import Input
from keras.optimizers import Adam
from matplotlib import pyplot as plt
from keras.models import Model
from keras import optimizers
from keras.applications.vgg16 import VGG16, preprocess_input


Now, upload the given dataset file shared with you in your google drive and give its path for the below given `project_path` variable. For example, a path is given below according to the file path in our google drive. You need to change this to match the path of yours.

In [0]:
project_path = "/content/drive/My Drive/AIML_Project/Dog_breed/"

Run the below code to extract all the images in the train.zip files given in the dataset. We are going to use these images as train and validation sets and their labels in further steps.

In [1]:
from zipfile import ZipFile
with ZipFile('train.zip', 'r') as z:
    z.extractall()

Repeat the same step for test.zip

In [3]:
with ZipFile('test.zip', 'r') as z:
    z.extractall()

Repeat the same step for sample_submission.csv.zip

In [5]:
with ZipFile("sample_submission.csv.zip","r") as z:
    z.extractall()

Repeat the same step for labels.csv.zip

In [6]:
with ZipFile("labels.csv.zip","r") as z:
    z.extractall()

After this process, we will have 4 files - Train folder, test folder and labels.csv and sample_submission.csv as part of your google drive

### Read labels.csv file using pandas

In [3]:
labels=pd.read_csv("labels.csv")

In [4]:
labels.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


### Print the count of each category of Dogs given in the dataset



In [5]:
labels["breed"].nunique()

120

In [6]:
len(labels)

10222

In [7]:
labels["breed"].value_counts()

scottish_deerhound                126
maltese_dog                       117
afghan_hound                      116
entlebucher                       115
bernese_mountain_dog              114
shih-tzu                          112
great_pyrenees                    111
pomeranian                        111
basenji                           110
samoyed                           109
tibetan_terrier                   107
airedale                          107
leonberg                          106
cairn                             106
beagle                            105
japanese_spaniel                  105
australian_terrier                102
blenheim_spaniel                  102
miniature_pinscher                102
irish_wolfhound                   101
lakeland_terrier                   99
saluki                             99
papillon                           96
siberian_husky                     95
whippet                            95
norwegian_elkhound                 95
pug         

In [8]:
breed=labels["breed"].value_counts()>=100

In [9]:
list_of_breed=list(breed[breed].index)

In [109]:
list_of_breed

['scottish_deerhound',
 'maltese_dog',
 'afghan_hound',
 'entlebucher',
 'bernese_mountain_dog',
 'shih-tzu',
 'pomeranian',
 'great_pyrenees',
 'basenji',
 'samoyed',
 'airedale',
 'tibetan_terrier',
 'leonberg',
 'cairn',
 'japanese_spaniel',
 'beagle',
 'australian_terrier',
 'blenheim_spaniel',
 'miniature_pinscher',
 'irish_wolfhound']

In [110]:
len(list_of_breed)

20

In [10]:
sub_labels=labels[labels["breed"].isin(list_of_breed)]

In [113]:
sub_labels

,id,breed
8,003df8b8a8b05244b1d920bb6cf451f9,basenji
9,0042188c895a2f14ef64a918ed9c7b64,scottish_deerhound
12,00693b8bc2470375cc744a6391d397ec,maltese_dog
29,00bee065dcec471f26394855c5c2f3de,cairn
48,013f8fdf6d638c7bb042f5f17e8a9fdc,tibetan_terrier
57,0162b3e0144fb1d1ab82fbff3ace9938,shih-tzu
63,0192d7174233ebdb3d720000413829ac,australian_terrier
70,01b36cb1b80ab8c3a7d2b7128ad21bdc,samoyed
79,01e787576c003930f96c966f9c3e1d44,scottish_deerhound
80,01ee3c7ff9bcaba9874183135877670e,entlebucher


In [114]:
len(sub_labels)

2184

In [115]:
sub_labels.nunique()

id       2184
breed      20
dtype: int64

In [116]:
sub_labels["breed"].value_counts()

scottish_deerhound      126
maltese_dog             117
afghan_hound            116
entlebucher             115
bernese_mountain_dog    114
shih-tzu                112
pomeranian              111
great_pyrenees          111
basenji                 110
samoyed                 109
airedale                107
tibetan_terrier         107
cairn                   106
leonberg                106
japanese_spaniel        105
beagle                  105
blenheim_spaniel        102
australian_terrier      102
miniature_pinscher      102
irish_wolfhound         101
Name: breed, dtype: int64

In [117]:
sub_labels.head()

,id,breed
8,003df8b8a8b05244b1d920bb6cf451f9,basenji
9,0042188c895a2f14ef64a918ed9c7b64,scottish_deerhound
12,00693b8bc2470375cc744a6391d397ec,maltese_dog
29,00bee065dcec471f26394855c5c2f3de,cairn
48,013f8fdf6d638c7bb042f5f17e8a9fdc,tibetan_terrier


### Get one-hot encodings of labels

In [121]:
#Doing Label Encoding for the "breed" column in the labels dataframe

In [11]:
encoder=LabelEncoder()

In [12]:
sub_labels["encoded"]=encoder.fit_transform(sub_labels["breed"])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [13]:
sub_labels.head()

,id,breed,encoded
8,003df8b8a8b05244b1d920bb6cf451f9,basenji,3
9,0042188c895a2f14ef64a918ed9c7b64,scottish_deerhound,17
12,00693b8bc2470375cc744a6391d397ec,maltese_dog,13
29,00bee065dcec471f26394855c5c2f3de,cairn,7
48,013f8fdf6d638c7bb042f5f17e8a9fdc,tibetan_terrier,19


In [13]:
ids=sub_labels["id"]

In [14]:
ids=ids.values

In [124]:
ids

array(['003df8b8a8b05244b1d920bb6cf451f9',
       '0042188c895a2f14ef64a918ed9c7b64',
       '00693b8bc2470375cc744a6391d397ec', ...,
       'ffcffab7e4beef9a9b8076ef2ca51909',
       'ffe2ca6c940cddfee68fa3cc6c63213f',
       'ffe5f6d8e2bff356e9482a80a6e29aac'], dtype=object)

In [125]:
len(ids)

2184

## Preparing training dataset
1. Write a code which reads each and every id from labels.csv file and loads the corresponding image (in RGB - 128, 128, 3) from the train folder. <br>
2. Create 2 variables <br> 
     a.  x_train - Should have all the images of the dogs from train folder <br>
     b.  y_train - Corresponding label of the dog <br>
<u>Note:</u> The id of the dog images and its corresponding labels are available in labels.csv file   
<u>Hint:</u> Watch the video shared on "Preparing the training dataset" if you face issue on creating the training dataset

In [15]:
img=cv2.imread("D://Project1_AfterResidency8_CNN//train//000bec180eb18c7604dcecc8fe0dba07.jpg")

In [16]:
img

array([[[  0,  46, 108],
        [ 42,  92, 152],
        [ 70, 120, 180],
        ...,
        [ 69, 144, 176],
        [ 82, 157, 189],
        [107, 182, 214]],

       [[ 47,  93, 151],
        [ 75, 121, 179],
        [ 64, 111, 167],
        ...,
        [ 56, 136, 167],
        [ 60, 139, 172],
        [ 77, 157, 188]],

       [[ 56,  98, 151],
        [ 94, 137, 188],
        [ 70, 113, 162],
        ...,
        [ 81, 168, 200],
        [ 75, 162, 196],
        [ 81, 168, 200]],

       ...,

       [[ 69, 122, 189],
        [ 66, 119, 186],
        [ 63, 116, 183],
        ...,
        [106, 160, 213],
        [107, 159, 212],
        [102, 156, 209]],

       [[ 66, 119, 186],
        [ 64, 117, 184],
        [ 63, 116, 183],
        ...,
        [ 88, 145, 197],
        [ 92, 146, 199],
        [ 90, 147, 199]],

       [[ 62, 115, 182],
        [ 61, 114, 181],
        [ 63, 116, 183],
        ...,
        [ 74, 132, 184],
        [ 81, 138, 190],
        [ 82, 140, 192]]

In [128]:
data=[]
label_of_data=[]

In [129]:
images=os.listdir("train")

In [130]:
for image in images:
    if image[:-4] in ids:
        img=load_img("D://Project1_AfterResidency8_CNN//train//"+image)
        img_arr=img_to_array(img)
        img_arr=cv2.resize(img_arr,(128,128))
        data.append(img_arr)
        label=sub_labels[sub_labels["id"]==image[:-4]]["encoded"].values[0]
        label_of_data.append(label)

In [131]:
len(label_of_data)

2184

In [132]:
len(data)

2184

In [133]:
data=np.array(data)

In [134]:
data.shape

(2184, 128, 128, 3)

In [135]:
data[0]

array([[[122.94702 , 130.58765 ,  66.04077 ],
        [134.09485 , 145.14954 ,  79.25891 ],
        [160.25952 , 167.85364 , 113.62671 ],
        ...,
        [134.26562 , 136.66296 ,  71.75    ],
        [150.26965 , 156.24487 ,  92.24487 ],
        [ 97.66626 , 109.76306 ,  48.864624]],

       [[178.57251 , 178.27563 , 122.16492 ],
        [121.38257 , 132.43726 ,  66.437256],
        [104.31348 , 120.03223 ,  51.9541  ],
        ...,
        [115.27002 , 115.784546,  55.784546],
        [142.05188 , 145.88782 ,  85.661255],
        [141.71362 , 149.46094 ,  89.46094 ]],

       [[142.6377  , 146.92712 ,  95.08301 ],
        [126.51123 , 141.92981 ,  77.28247 ],
        [ 93.13049 , 115.39612 ,  43.92737 ],
        ...,
        [138.79175 , 147.05737 ,  88.58862 ],
        [149.63684 , 158.41919 ,  99.153564],
        [ 94.63953 , 105.17078 ,  43.90515 ]],

       ...,

       [[158.76416 , 184.89624 , 108.95093 ],
        [185.66064 , 199.5824  , 141.48987 ],
        [122.23865 , 1

Normalize the training data and convert into 4 dimensions so that it can be used as an input to conv layers in the model

In [136]:
data=data/255

In [137]:
data[0]

array([[[0.4821452 , 0.51210845, 0.2589834 ],
        [0.52586216, 0.56921387, 0.31081927],
        [0.6284687 , 0.65824956, 0.44559494],
        ...,
        [0.5265319 , 0.5359332 , 0.28137255],
        [0.58929276, 0.612725  , 0.3617446 ],
        [0.38300493, 0.43044338, 0.19162598]],

       [[0.70028436, 0.69912016, 0.4790781 ],
        [0.47601008, 0.5193618 , 0.26053825],
        [0.40907246, 0.47071463, 0.20374158],
        ...,
        [0.4520393 , 0.45405704, 0.21876292],
        [0.5570662 , 0.5721091 , 0.3359265 ],
        [0.5557397 , 0.5861213 , 0.35082722]],

       [[0.5593635 , 0.5761848 , 0.37287453],
        [0.49612248, 0.5565875 , 0.30306852],
        [0.36521763, 0.4525338 , 0.17226419],
        ...,
        [0.54428136, 0.57669556, 0.34740636],
        [0.5868111 , 0.6212517 , 0.38883752],
        [0.37113538, 0.41243443, 0.17217706]],

       ...,

       [[0.62260455, 0.7250833 , 0.42725855],
        [0.728081  , 0.78267604, 0.5548622 ],
        [0.47936726, 0

### Split the training and validation data from `x_train_data` and `y_train_data` obtained from above step

In [138]:
x_train,x_test,y_train,y_test=train_test_split(data,label_of_data,test_size=0.30)

In [139]:
x_train

array([[[[0.6456428 , 0.67701536, 0.7201526 ],
         [0.6358044 , 0.6632554 , 0.7018986 ],
         [0.6332108 , 0.64956534, 0.6747214 ],
         ...,
         [0.623741  , 0.6551136 , 0.7296234 ],
         [0.5890319 , 0.6204044 , 0.6949142 ],
         [0.5882353 , 0.61960787, 0.70980394]],

        [[0.62961566, 0.6626637 , 0.7106407 ],
         [0.618075  , 0.6444259 , 0.6887858 ],
         [0.6230392 , 0.6387255 , 0.67626953],
         ...,
         [0.5961397 , 0.6275123 , 0.7020221 ],
         [0.5780015 , 0.60937405, 0.68388385],
         [0.581924  , 0.61329657, 0.70349264]],

        [[0.60793793, 0.64323205, 0.7020556 ],
         [0.598674  , 0.62757355, 0.6859289 ],
         [0.607937  , 0.63147885, 0.6784314 ],
         ...,
         [0.58253676, 0.6139093 , 0.6884191 ],
         [0.5814558 , 0.6128284 , 0.6873382 ],
         [0.5959559 , 0.62732846, 0.7175245 ]],

        ...,

        [[0.3050542 , 0.23054439, 0.1093434 ],
         [0.51259476, 0.45111442, 0.30573204]

In [140]:
y_train

[19,
 12,
 6,
 7,
 11,
 6,
 9,
 8,
 19,
 3,
 0,
 4,
 5,
 6,
 9,
 11,
 13,
 10,
 17,
 10,
 8,
 16,
 8,
 12,
 16,
 7,
 1,
 2,
 13,
 11,
 4,
 12,
 2,
 2,
 3,
 1,
 8,
 18,
 8,
 17,
 3,
 17,
 13,
 11,
 14,
 10,
 19,
 0,
 6,
 17,
 18,
 9,
 14,
 7,
 3,
 17,
 7,
 1,
 1,
 12,
 0,
 19,
 8,
 6,
 7,
 16,
 0,
 15,
 0,
 7,
 4,
 17,
 3,
 12,
 15,
 19,
 2,
 7,
 10,
 5,
 15,
 18,
 18,
 14,
 5,
 19,
 7,
 7,
 11,
 4,
 16,
 14,
 15,
 7,
 5,
 15,
 8,
 16,
 2,
 12,
 15,
 8,
 5,
 3,
 6,
 3,
 14,
 12,
 12,
 12,
 4,
 12,
 7,
 15,
 19,
 5,
 0,
 13,
 19,
 12,
 10,
 11,
 15,
 17,
 12,
 12,
 9,
 6,
 18,
 10,
 19,
 4,
 4,
 2,
 5,
 2,
 1,
 3,
 13,
 19,
 5,
 15,
 9,
 2,
 14,
 18,
 4,
 0,
 4,
 6,
 8,
 6,
 14,
 10,
 17,
 3,
 17,
 18,
 14,
 8,
 7,
 6,
 15,
 9,
 9,
 16,
 10,
 4,
 6,
 15,
 5,
 8,
 19,
 15,
 6,
 15,
 9,
 13,
 4,
 11,
 13,
 14,
 19,
 5,
 17,
 5,
 17,
 14,
 3,
 19,
 13,
 3,
 12,
 17,
 2,
 5,
 12,
 3,
 11,
 8,
 19,
 5,
 4,
 5,
 3,
 0,
 9,
 2,
 14,
 19,
 5,
 2,
 1,
 8,
 17,
 15,
 5,
 13,
 13,
 14,
 8,
 4,
 10,

In [141]:
y_train=np.array(y_train)
y_test=np.array(y_test)

In [142]:
y_train.shape

(1528,)

In [143]:
y_train=to_categorical(y_train,20)
y_test=to_categorical(y_test,20)

In [144]:
y_train.shape

(1528, 20)

In [145]:
y_train[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1.], dtype=float32)

In [146]:
y_test.shape

(656, 20)

In [147]:
y_test[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 0.], dtype=float32)

### Loading the test data
Read the id column from the samples_submission.csv and store it in test_img

In [0]:
# No Need as the test images cannot be used for this exercise as labels for the test images
# is not known

Run the below code to load the test image files in x_test_feature

In [0]:
x_test_feature = []
i = 0 # initialisation
for f in tqdm(test_img.values): # f for format ,jpg
    img = cv2.imread('./test/{}.jpg'.format(f), 0)
    img_resize = cv2.resize(img, (img_rows, img_cols)) 
    x_test_feature.append(img_resize)

Normalize the test data and convert it into 4 dimensions

In [0]:
# No Need to work on test data 

### Build a basic conv neural network with 2 conv layers (kernel sizes - 5 and 3) add layers as mentioned below for classification.

1. Add a Dense layer with 256 neurons with `relu` activation

2. Add a Dense layer with 120 neurons as final layer (as there are 120 classes in the given dataset) with `softmax` activation for classifiaction. 

In [154]:
model1=Sequential()

#1st Convolutional Layer
model1.add(Conv2D(32,(3,3),padding="same",input_shape=(128,128,3)))
model1.add(Activation("relu"))

#2nd Convolutional Layer
model1.add(Conv2D(64,(5,5),padding="same"))
model1.add(Activation("relu"))

#Adding MaxPool Layer
model1.add(MaxPooling2D(pool_size=(2,2)))

#Flattening the Output from Convolutional Layer
model1.add(Flatten())

#Add 1st Dense Layer
model1.add(Dense(256))
model1.add(Activation("relu"))


#Add the Output Layer
model1.add(Dense(20))
model1.add(Activation("softmax"))

#Adding Optimizer
optimizer=Adam(learning_rate=0.001)

#Compile Model
model1.compile(loss="categorical_crossentropy",optimizer=optimizer,metrics=["accuracy"])

In [155]:
model1.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 128, 128, 32)      896       
_________________________________________________________________
activation_9 (Activation)    (None, 128, 128, 32)      0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 128, 128, 64)      51264     
_________________________________________________________________
activation_10 (Activation)   (None, 128, 128, 64)      0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 64, 64, 64)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 262144)            0         
_________________________________________________________________
dense_5 (Dense)              (None, 256)              

### Use batch_size = 128 and epochs = 10 and execute the model

In [156]:
model1.fit(x_train,y_train,
          validation_data=(x_test,y_test),
          epochs=10,
          batch_size=128)

Train on 1528 samples, validate on 656 samples
Epoch 1/10
1528/1528 [==============================] - 399s 261ms/step - loss: 5.7610 - accuracy: 0.0504 - val_loss: 2.9959 - val_accuracy: 0.0442
Epoch 2/10
1528/1528 [==============================] - 371s 243ms/step - loss: 2.9917 - accuracy: 0.0870 - val_loss: 2.9900 - val_accuracy: 0.0610
Epoch 3/10
1528/1528 [==============================] - 390s 255ms/step - loss: 2.9083 - accuracy: 0.1505 - val_loss: 2.9342 - val_accuracy: 0.1067
Epoch 4/10
1528/1528 [==============================] - 378s 247ms/step - loss: 2.4711 - accuracy: 0.3220 - val_loss: 2.9273 - val_accuracy: 0.1235
Epoch 5/10
1528/1528 [==============================] - 370s 242ms/step - loss: 1.6955 - accuracy: 0.5432 - val_loss: 3.4379 - val_accuracy: 0.1052
Epoch 6/10
1528/1528 [==============================] - 408s 267ms/step - loss: 0.8850 - accuracy: 0.7670 - val_loss: 4.0183 - val_accuracy: 0.1037
Epoch 7/10
1528/1528 [==============================] - 387s 253m

#The model accuracy is very poor !!!!

### Use Data Augmentation in the above model to see if the accuracy improves


In [157]:
datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

In [158]:
train_generator=datagen.flow(x_train,y_train,batch_size=32)

In [159]:
test_generator=datagen.flow(x_test,y_test,batch_size=32)

### Using the above objects, create the image generators with variable names `train_generator` and `val_generator`

You need to use train_datagen.flow() and val_datagen.flow()

In [0]:
# created above and used in above step

### Fit the model using fit_generator() using `train_generator` and `val_generator` from the above step with 10 epochs

In [162]:
model1.fit_generator(train_generator,
                     steps_per_epoch=x_train.shape[0]/2,
                     epochs=10,
                     validation_data=test_generator)

Epoch 1/10
764/764 [==============================] - 3332s 4s/step - loss: 2.9936 - accuracy: 0.0583 - val_loss: 3.0172 - val_accuracy: 0.0412
Epoch 2/10
764/764 [==============================] - 3301s 4s/step - loss: 2.9935 - accuracy: 0.0584 - val_loss: 3.0078 - val_accuracy: 0.0412
Epoch 3/10
764/764 [==============================] - 3286s 4s/step - loss: 2.9935 - accuracy: 0.0581 - val_loss: 2.9996 - val_accuracy: 0.0412
Epoch 4/10
764/764 [==============================] - 3298s 4s/step - loss: 2.9935 - accuracy: 0.0580 - val_loss: 2.9814 - val_accuracy: 0.0412
Epoch 5/10
764/764 [==============================] - 3286s 4s/step - loss: 2.9933 - accuracy: 0.0585 - val_loss: 3.0158 - val_accuracy: 0.0412
Epoch 6/10
764/764 [==============================] - 3277s 4s/step - loss: 2.9935 - accuracy: 0.0579 - val_loss: 2.9848 - val_accuracy: 0.0412
Epoch 7/10
764/764 [==============================] - 3268s 4s/step - loss: 2.9932 - accuracy: 0.0585 - val_loss: 3.0271 - val_accuracy:

# Model Accuracy is Still Poor. So buiding a complete VGG Model to improve the model Accuracy

In [17]:
vgg_data=[]
vgg_label_of_data=[]

In [18]:
images=os.listdir("train")

In [19]:
for image in images:
    if image[:-4] in ids:
        img=load_img("D://Project1_AfterResidency8_CNN//train//"+image)
        img_arr=img_to_array(img)
        img_arr=cv2.resize(img_arr,(224,224))
        vgg_data.append(img_arr)
        label=sub_labels[sub_labels["id"]==image[:-4]]["encoded"].values[0]
        vgg_label_of_data.append(label)

In [20]:
vgg_data=np.array(vgg_data)

In [21]:
vgg_data.shape

(2184, 224, 224, 3)

In [22]:
vgg_data=vgg_data/255

In [23]:
x_train_vgg,x_test_vgg,y_train_vgg,y_test_vgg=train_test_split(vgg_data,vgg_label_of_data,test_size=0.30)

In [24]:
y_train_vgg=np.array(y_train_vgg)
y_test_vgg=np.array(y_test_vgg)

In [25]:
y_train_vgg=to_categorical(y_train_vgg,20)
y_test_vgg=to_categorical(y_test_vgg,20)

In [26]:
y_train_vgg.shape

(1528, 20)

In [27]:
y_test_vgg.shape

(656, 20)

In [27]:
model_VGG=Sequential()

#1st Block
model_VGG.add(Conv2D(input_shape=(224,224,3),filters=64,kernel_size=(3,3),
                     padding="same",activation="relu"))
model_VGG.add(Conv2D(filters=64,kernel_size=(3,3),padding="same",activation="relu"))
model_VGG.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))

#2nd Block
model_VGG.add(Conv2D(filters=128,kernel_size=(3,3),padding="same",activation="relu"))
model_VGG.add(Conv2D(filters=128,kernel_size=(3,3),padding="same",activation="relu"))
model_VGG.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))

#3rd Block
model_VGG.add(Conv2D(filters=256,kernel_size=(3,3),padding="same",activation="relu"))
model_VGG.add(Conv2D(filters=256,kernel_size=(3,3),padding="same",activation="relu"))
model_VGG.add(Conv2D(filters=256,kernel_size=(3,3),padding="same",activation="relu"))
model_VGG.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))

#4th Block
model_VGG.add(Conv2D(filters=512,kernel_size=(3,3),padding="same",activation="relu"))
model_VGG.add(Conv2D(filters=512,kernel_size=(3,3),padding="same",activation="relu"))
model_VGG.add(Conv2D(filters=512,kernel_size=(3,3),padding="same",activation="relu"))
model_VGG.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))

#5th Block
model_VGG.add(Conv2D(filters=512,kernel_size=(3,3),padding="same",activation="relu"))
model_VGG.add(Conv2D(filters=512,kernel_size=(3,3),padding="same",activation="relu"))
model_VGG.add(Conv2D(filters=512,kernel_size=(3,3),padding="same",activation="relu"))
model_VGG.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))

#Add Fully Connected Layer
model_VGG.add(Flatten())
model_VGG.add(Dense(units=4096,activation="relu"))
model_VGG.add(Dense(units=4096,activation="relu"))
model_VGG.add(Dense(units=20,activation="softmax"))
optimizer=Adam(lr=0.001)
model_VGG.compile(loss="categorical_crossentropy",optimizer=optimizer,metrics=["accuracy"])


Instructions for updating:
Colocations handled automatically by placer.


In [28]:
model_VGG.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 112, 112, 128)     147584    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 56, 56, 128)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 56, 56, 256)      

In [29]:
model_VGG.fit(x_train_vgg,y_train_vgg,
          validation_data=(x_test_vgg,y_test_vgg),
          epochs=5,
          batch_size=50)

Instructions for updating:
Use tf.cast instead.
Train on 1528 samples, validate on 656 samples
Epoch 1/5
1528/1528 [==============================] - 3647s 2s/step - loss: 3.0165 - accuracy: 0.0491 - val_loss: 2.9977 - val_accuracy: 0.0427
Epoch 2/5
1528/1528 [==============================] - 3666s 2s/step - loss: 2.9960 - accuracy: 0.0543 - val_loss: 2.9974 - val_accuracy: 0.0610
Epoch 3/5
1528/1528 [==============================] - 3742s 2s/step - loss: 2.9942 - accuracy: 0.0517 - val_loss: 3.0004 - val_accuracy: 0.0610
Epoch 4/5
1528/1528 [==============================] - 3759s 2s/step - loss: 2.9940 - accuracy: 0.0563 - val_loss: 3.0002 - val_accuracy: 0.0610
Epoch 5/5
1528/1528 [==============================] - 3722s 2s/step - loss: 2.9938 - accuracy: 0.0537 - val_loss: 3.0025 - val_accuracy: 0.0610


# Model accuracy is still poor!!!

### Lets use Transfer Learning

Download the vgg wieght file from here : https://github.com/MinerKasch/applied_deep_learning/blob/master/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5

Use the below code to load VGG16 weights trained on ImageNet

In [0]:
# Instantiate the model with the pre-trained weights (no top)
base_model= VGG16(weights=(project_path+'vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'),
                 include_top=False, pooling='avg')

In [52]:
from keras.applications.vgg16 import VGG16, preprocess_input
# Instantiate the model with the pre-trained weights (no top)
base_model= VGG16(weights="imagenet",include_top=False,pooling="avg")

Print the summary of the base_model

In [53]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0     

### Add the following classification layers to the imported VGG Model <br>
1. Flatten Layer
2. Dense layer with 1024 neurons with activation as Relu
3. Dense layer with 256 neurons with activation as Relu
4. Dense layer with 120 neurons with activation as Softmax

In [61]:
inpt = Input(shape=(224,224,3),name = 'image_input')

In [62]:
vgg16=base_model(inpt)

In [63]:
x=Dense(1024,activation='relu',name="fc1")(vgg16)

In [64]:
x=Dense(256,activation="relu",name="fc2")(x)

In [65]:
x = Dense(20, activation='softmax', name='predictions')(x)


In [68]:
my_model = Model(input=inpt, output=x)


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("im..., outputs=Tensor("pr...)`
  """Entry point for launching an IPython kernel.


In [69]:
my_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image_input (InputLayer)     (None, 224, 224, 3)       0         
_________________________________________________________________
vgg16 (Model)                (None, 512)               14714688  
_________________________________________________________________
dense_6 (Dense)              (None, 1024)              525312    
_________________________________________________________________
fc2 (Dense)                  (None, 256)               262400    
_________________________________________________________________
predictions (Dense)          (None, 20)                5140      
Total params: 15,507,540
Trainable params: 15,507,540
Non-trainable params: 0
_________________________________________________________________


In [72]:
my_model.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])

In [73]:
my_model.fit(x_train_vgg,y_train_vgg,
          validation_data=(x_test_vgg,y_test_vgg),
          epochs=5,
          batch_size=50)

Train on 1528 samples, validate on 656 samples
Epoch 1/5
1528/1528 [==============================] - 3238s 2s/step - loss: 3.0358 - accuracy: 0.0360 - val_loss: 3.0240 - val_accuracy: 0.0457
Epoch 2/5
1528/1528 [==============================] - 3189s 2s/step - loss: 3.0029 - accuracy: 0.0478 - val_loss: 3.0013 - val_accuracy: 0.0549
Epoch 3/5
1528/1528 [==============================] - 3214s 2s/step - loss: 2.9862 - accuracy: 0.0700 - val_loss: 2.9893 - val_accuracy: 0.0701
Epoch 4/5
1528/1528 [==============================] - 3250s 2s/step - loss: 2.9752 - accuracy: 0.0857 - val_loss: 2.9800 - val_accuracy: 0.0823
Epoch 5/5
1528/1528 [==============================] - 3519s 2s/step - loss: 2.9650 - accuracy: 0.1145 - val_loss: 2.9719 - val_accuracy: 0.0854


### Make all the layers in the base_model (VGG16) to be non-trainable

In [30]:
base_model_again= VGG16(weights="imagenet",include_top=True,pooling="avg")

Instructions for updating:
Colocations handled automatically by placer.


In [31]:
base_model_again.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [32]:
X=base_model_again.layers[-2].output

In [33]:
predictions=Dense(20,activation="softmax")(X)

In [34]:
model_final=Model(inputs=base_model_again.input,output=predictions)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  """Entry point for launching an IPython kernel.


In [35]:
model_final.compile(loss = "categorical_crossentropy", 
                    optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), 
                    metrics=["accuracy"])

### Fit and compile the model with batch_size = 128 and epochs = 10 and execute the model

Try to get training and validation accuracy to be more than 90%

In [36]:
model_final.fit(x_train_vgg,y_train_vgg,
          validation_data=(x_test_vgg,y_test_vgg),
          epochs=5,
          batch_size=32)

Instructions for updating:
Use tf.cast instead.
Train on 1528 samples, validate on 656 samples
Epoch 1/5
1528/1528 [==============================] - 2891s 2s/step - loss: 3.0734 - accuracy: 0.0733 - val_loss: 2.9526 - val_accuracy: 0.1448
Epoch 2/5
1528/1528 [==============================] - 2907s 2s/step - loss: 2.7950 - accuracy: 0.1545 - val_loss: 2.6816 - val_accuracy: 0.1616
Epoch 3/5
1528/1528 [==============================] - 2872s 2s/step - loss: 2.3846 - accuracy: 0.2801 - val_loss: 2.1469 - val_accuracy: 0.3750
Epoch 4/5
1528/1528 [==============================] - 2853s 2s/step - loss: 1.8673 - accuracy: 0.4012 - val_loss: 1.7339 - val_accuracy: 0.4543
Epoch 5/5
1528/1528 [==============================] - 2867s 2s/step - loss: 1.4002 - accuracy: 0.5445 - val_loss: 1.3942 - val_accuracy: 0.5564


# Observation:

Assumption: Because of hardware issues, I am training model on just 20 classes inspite of 120
            classes wherein each class has maximum of 120 samples.Also, only 5 epochs are run
            for each model

1. When I made my model learn all weights of feature extraction layers as well as classification
   layers, my model accuracy was just 4% 
2. When I did transfer learning taking weights of feature extraction layers and learning 
   the classification layers, my model accuracy increased to 8%
3. But when I did transfer learning taking weights of feature extraction as well as classification
   layers, my model accuracy increased to 55%
